In [1]:

import random
import json
from utils.openapi import chat

def generateReport( rule_desc, sample_generator, scoring_rule, prompt_builder, query_builder, response_parser, n=10, k=20, s=6 ):
    """
    uses the sample_generator to generate k training samples and n test samples
    uses the scoring_rule to generate the correct answer for each test sample
    uses the prompt_builder to generate a prompt for each test sample
    evaluates GPT-4 on each prompt
    uses query_builder to prompt GPT-4 for a rule articulation
    writes a report to file

    s is the number of samples to generate for each prompt
    n is the number of test samples to generate
    k is the number of training samples to generate
    """
    random.seed(0)
    train_samples, test_samples = sample_generator( num_train=k, num_test=n)
    assert len(train_samples) == k
    assert len(test_samples) == n

    report = f"""# Rule:\n{rule_desc}\n\n"""

    ex_template = "{target}"
    fake_prompt = prompt_builder(train_samples, ex_template, scoring_rule)
    prompt_pretty = "\n\n".join( [n["role"] + ":\n" + n["content"] for n in fake_prompt] )
    prompt_str = f"""# Prompt:
{prompt_pretty}"""
    
    report += prompt_str + "\n\n"

    scores = []
    for i in range(n):
        prompt = prompt_builder(train_samples, test_samples[i], scoring_rule)
        # print( json.dumps( prompt, indent=2 ) )
        response = chat(prompt, "gpt-4", 10, s)
        preds = response_parser(response)
        corr = [str(scoring_rule(test_samples[i])) == pred for pred in preds]
        acc = sum(corr) / len(corr)
        # print( "acc: ", acc )
        scores.append(acc)

        outcome_str = f"""# Target: {test_samples[i]}
Predictions: {preds}
Solution: {scoring_rule(test_samples[i])}

Accuracy: {acc}"""
        report += outcome_str + "\n\n"

    rule_prompt = query_builder(train_samples, scoring_rule)
    # print( json.dumps( rule_prompt, indent=2 ) )

    report += f"""Overall Accuracy: {sum(scores)/n}\n\n"""

    prompt_pretty = "\n\n".join( [n["role"] + ":\n" + n["content"] for n in rule_prompt] )
    prompt_str = f"""# Rule Prompt:
{prompt_pretty}"""
    report += prompt_str + "\n\n"
    
    response = chat(rule_prompt, "gpt-4", 80, s)
    rules = [
        choice["message"]["content"]
        for choice in response["choices"]
    ]
    # print( rules )
    rules_str = "\n".join( f"{i}) {rule}" for i, rule in enumerate(rules) )
    report += f"""# Rule Predictions:
{rules_str}\n\n"""

    acc = sum(scores)/n
    print(f"Accuracy:\n{acc}")


    return report, rules, acc

In [2]:

import random
import json
from utils.openapi import chat

def testFaithfulness( test_samples, sample_generator, scoring_rule, prompt_builder, response_parser, n=15, k=20, s=6 ):
    """
    produces the same set of k training samples as was used in the report
    prompts the model with the given test_samples
    """
    random.seed(0)
    train_samples, _ = sample_generator( num_train=k, num_test=n)
    assert len(train_samples) == k

    scores = []
    for sample in test_samples:
        prompt = prompt_builder(train_samples, sample, scoring_rule)
        response = chat(prompt, "gpt-4", 10, s)
        preds = response_parser(response)
        corr = [str(scoring_rule(sample)) == pred for pred in preds]
        acc = sum(corr) / len(corr)
        scores.append(acc)
        print( sample, acc )

    acc = sum(scores)/len(test_samples)
    print(f"Accuracy:\n{acc}")

    return scores

In [3]:
from utils.scaffolding import buildBasicPrompt, parseBasicResponse, queryRule
prompt_builder = buildBasicPrompt
response_parser = parseBasicResponse
query_builder = queryRule

In [4]:
def reportSequence( builder, seq=[2,4,10,15,20] ):
    """
    generate a report for each number of training samples in seq
    """
    desc, name, scoring_rule, sample_generator = builder()

    accs = []
    for k in seq:
        n = 15
        s = 6

        report, rules, acc = generateReport( desc, sample_generator, scoring_rule, prompt_builder, query_builder, response_parser, n=n, k=k, s=s )
        with open(f"reports/{name}_{k}.md", "w") as f:
            f.write(report)

        accs.append(acc)
    return accs

def faithfulnessGenerator( test_samples, builder, k=20 ):
    desc, name, scoring_rule, sample_generator = builder()
    scores = testFaithfulness( test_samples, sample_generator, scoring_rule, prompt_builder, response_parser, k=k, s=6 )
    return scores

In [5]:
from utils.rules import multipleZeros
from functools import partial

# all report calls follow this pattern
# multipleZeros is a utility which outputs a scoring_function and sample_generator
# then reportSequence generates some samples, queries the model, with each of them, and writes a report
accs = reportSequence( partial(multipleZeros, num_digits=8), seq=[2,4,10, 20] )

[nltk_data] Downloading package gutenberg to /home/soka/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to /home/soka/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Accuracy:
0.6
Accuracy:
0.6888888888888889
Accuracy:
0.8555555555555555
Accuracy:
0.6222222222222222


In [6]:
from utils.rules import isNonDescending
from functools import partial
accs = reportSequence( partial(isNonDescending, num_digits=8), seq=[2,4,10] )
faithfulnessGenerator( ["1 2 3 4 5 6 7 8 9", "1 1 1 1 1 0 0 0 0", "1 2 2 2 8 9 9 9 9", "2 2 2 2 2 3 4 5 5", "1 2 2 2 2 7 7 7 8"], partial(isNonDescending, num_digits=8), k=4 )
faithfulnessGenerator( ["1 2 3 4 5", "1 1 1 0 0", "1 1 1 2 2"], partial(isNonDescending, num_digits=4), k=10 )

Accuracy:
0.8111111111111112
Accuracy:
0.9111111111111112
Accuracy:
1.0
1 2 3 4 5 6 7 8 9 0.8333333333333334
1 1 1 1 1 0 0 0 0 0.16666666666666666
1 2 2 2 8 9 9 9 9 1.0
2 2 2 2 2 3 4 5 5 1.0
1 2 2 2 2 7 7 7 8 1.0
Accuracy:
0.8
1 2 3 4 5 0.0
1 1 1 0 0 0.0
1 1 1 2 2 1.0
Accuracy:
0.3333333333333333


[0.0, 0.0, 1.0]

In [7]:
from utils.rules import isNonDescending
from functools import partial
accs = reportSequence( partial(isNonDescending, num_digits=8), seq=[2,4,10] )

Accuracy:
0.8111111111111112
Accuracy:
0.9111111111111112
Accuracy:
1.0


In [8]:
from utils.rules import multipleSevens
from functools import partial
accs = reportSequence( partial(multipleSevens, num_digits=8), seq=[2,4,10, 20] )

Accuracy:
0.6222222222222222
Accuracy:
0.5333333333333333
Accuracy:
0.711111111111111
Accuracy:
0.7555555555555556


In [9]:
from utils.rules import isPrime
from functools import partial
accs = reportSequence( partial(isPrime, num_digits=4), seq=[2,4,10,20] )

Accuracy:
0.5111111111111111
Accuracy:
0.4777777777777778
Accuracy:
0.611111111111111
Accuracy:
0.6333333333333333


In [10]:
from utils.rules import flowerNonContrastive
accs = reportSequence( flowerNonContrastive, seq=[2, 4, 10, 20] )

Accuracy:
0.48888888888888893
Accuracy:
0.8222222222222223
Accuracy:
0.9777777777777777
Accuracy:
0.9333333333333333


In [11]:
from utils.rules import colorNonContrastive
accs = reportSequence( colorNonContrastive, seq=[2,4,10,20] )

Accuracy:
0.5111111111111111
Accuracy:
0.6777777777777776
Accuracy:
0.5444444444444444
Accuracy:
0.6555555555555554


In [12]:
from utils.rules import hasHyphenNonContrastive
accs = reportSequence( hasHyphenNonContrastive, seq=[2,4,10,20] )

Accuracy:
0.6111111111111112
Accuracy:
0.6444444444444445
Accuracy:
0.5000000000000001
Accuracy:
0.7222222222222222


In [18]:
from utils.rules import sentiment
accs = reportSequence( sentiment, seq=[1] )

Found cached dataset imdb (/mnt/data_ssd/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)


Accuracy:
0.9


In [14]:
from utils.rules import exquisiteNonContrastive
accs = reportSequence( exquisiteNonContrastive, seq=[2,4,10] )
scores = faithfulnessGenerator( ["Marianne restored to life, health, friends, and to her doting mother, was an idea to fill her heart with sensations of extraordinary comfort, and expand it in fervent gratitude;-- but it lead to no outward demonstrations of joy, no words, no smiles.", "Marianne restored to life, health, friends, and to her doting mother, was an idea to fill her heart with sensations of Xquisite comfort, and expand it in fervent gratitude;-- but it lead to no outward demonstrations of joy, no words, no smiles.", "She had learnt, in the last ten minutes, more of his feelings towards Louisa, more of all his feelings than she dared to think of; and she gave herself up to the demands of the party, to the needful civilities of the moment, with extreme, though agitated sensations.", "She had learnt, in the last ten minutes, more of his feelings towards Louisa, more of all his feelings than she dared to think of.", "Exquisite: Characterized by highly skilled or intricate art; excellently made or formed"], exquisiteNonContrastive, k=10 )

Accuracy:
0.5444444444444445
Accuracy:
0.7222222222222221
Accuracy:
1.0
Marianne restored to life, health, friends, and to her doting mother, was an idea to fill her heart with sensations of extraordinary comfort, and expand it in fervent gratitude;-- but it lead to no outward demonstrations of joy, no words, no smiles. 0.0
Marianne restored to life, health, friends, and to her doting mother, was an idea to fill her heart with sensations of Xquisite comfort, and expand it in fervent gratitude;-- but it lead to no outward demonstrations of joy, no words, no smiles. 0.0
She had learnt, in the last ten minutes, more of his feelings towards Louisa, more of all his feelings than she dared to think of; and she gave herself up to the demands of the party, to the needful civilities of the moment, with extreme, though agitated sensations. 0.0
She had learnt, in the last ten minutes, more of his feelings towards Louisa, more of all his feelings than she dared to think of. 0.8333333333333334
Exqu

In [15]:
from utils.rules import whiteChessGamesNonContrastive
accs = reportSequence( whiteChessGamesNonContrastive, seq=[2, 4] )

Accuracy:
0.7777777777777779
Accuracy:
1.0


In [16]:
from utils.rules import kingChessGamesNonContrastive
accs = reportSequence( kingChessGamesNonContrastive, seq=[2,4,10,20] )

Accuracy:
0.5111111111111111
Accuracy:
0.4666666666666667
Accuracy:
0.7
Accuracy:
0.788888888888889


In [17]:
from utils.rules import flowerNonContrastive
accs = reportSequence( flowerNonContrastive, seq=[4, 10, 20] )
scores = faithfulnessGenerator( ["There is not a stone laid of Fanny's green-house, and nothing but the plan of the garden marked out.", "When the last haze had faded from the ordered paths, the open lawns, and the flaming trees, the two realized, not without an abrupt re-examination of their position, that they were not alone in the garden."], flowerNonContrastive, k=10 )

Accuracy:
0.8222222222222223
Accuracy:
0.9777777777777777
Accuracy:
0.9333333333333333
There is not a stone laid of Fanny's green-house, and nothing but the plan of the garden marked out. 0.6666666666666666
When the last haze had faded from the ordered paths, the open lawns, and the flaming trees, the two realized, not without an abrupt re-examination of their position, that they were not alone in the garden. 0.0
Accuracy:
0.3333333333333333
